# Pre-processing books




In [ ]:
#!pip list
#haystack: 2.0.1 -> 2.13
#pydantic: 1.10.7 -> 2.11.5
# pypdf-4.3.1 -> 5.5.0 # pypdf-3.8.1 -> 3.9.1
# sentence-transformers -> 2.2.2 -> pip install --upgrade sentence-transformers==4.0.1
#torch ->  2.2.0 -> pip uninstall, install: torch-2.2.2
#tensorflow -> 2.12.0 -> 2.16.2
#Successfully installed tf-keras-2.16.0

In [ ]:
!pip uninstall -y sentence-transformers huggingface-hub transformers haystack-ai torch torchvision torchaudio
!pip cache purge

!pip install scikit-learn pandas pymupdf pytesseract
!apt-get update
!apt-get install -y tesseract-ocr
!apt-get install -y tesseract-ocr-spa

Found existing installation: sentence-transformers 4.1.0
Uninstalling sentence-transformers-4.1.0:
  Successfully uninstalled sentence-transformers-4.1.0
Found existing installation: huggingface-hub 0.33.4
Uninstalling huggingface-hub-0.33.4:
  Successfully uninstalled huggingface-hub-0.33.4
Found existing installation: transformers 4.53.2
Uninstalling transformers-4.53.2:
  Successfully uninstalled transformers-4.53.2
Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Files removed: 0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 77.3 MB/s eta 0:00:00
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archi

In [ ]:
import os
os.environ['TESSDATA_PREFIX'] = '/usr/share/tesseract-ocr/4.00/tessdata/'

# Carga de metadatos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

csv_path = '/content/drive/MyDrive/rag_project/articles/pueblos_y_nacionalidades.csv'
metadata = pd.read_csv(csv_path, sep=';')
metadata['titulo'] = metadata['titulo'].str.replace('\n', ' ', regex=True)
print(metadata.head(2))

                   titulo      lengua  \
0  NACIONALIDAD TSA´CHILA    Tsa´Fiki   
1  PUEBLO AFROECUATORIANO  Castellano   

                                        ubicacion  
0                SANTO DOMINGO DE LOS TSÁ^CHILAS   
1  REGIÓN COSTA / REGIÓN SIERRA / REGIÓN AMAZONÍA  


# Obtener rutas de archivos

In [ ]:
import glob

def get_txt_paths(folder_path):
    return glob.glob(f"{folder_path}/*.txt")

folder = "/content/drive/MyDrive/rag_project/articles"
txt_files = get_txt_paths(folder)

In [ ]:
print(f"Archivos TXT encontrados: {txt_files}")

Archivos TXT encontrados: []


# Extraer texto

In [ ]:
import pytesseract
from PIL import Image
import fitz  # PyMuPDF
import os

pdf_folder = "/content/drive/MyDrive/rag_project/articles"
output_folder = "/content/drive/MyDrive/rag_project/texts"
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(pdf_folder):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder, filename)
        doc = fitz.open(pdf_path)
        text = ""

        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            pix = page.get_pixmap()
            img_path = f"/tmp/page_{page_num}.png"
            pix.save(img_path)

            # OCR en imagen de la página
            text += pytesseract.image_to_string(Image.open(img_path), lang='spa') + "\n"

        doc.close()

        print(f"Archivo {filename} procesado, longitud texto: {len(text)}")

        txt_filename = filename.replace(".pdf", ".txt")
        with open(os.path.join(output_folder, txt_filename), "w", encoding="utf-8") as f:
            f.write(text)

Archivo ejemplo 2 de pueblos y nacionalidades.pdf procesado, longitud texto: 1472
Archivo ejemplo pueblos y nacionalidades.pdf procesado, longitud texto: 2349
Archivo EJEMPLO 1 DE PUEBLOS Y NACIONALIDADES.pdf procesado, longitud texto: 1345
Archivo EJEMPLO 2 DE PUEBLOS Y NACIONALIDADES.pdf procesado, longitud texto: 1279
Archivo EJEMPLO 3 DE PUEBLOS Y NACIONALIDADES.pdf procesado, longitud texto: 1769
Archivo ejemplo 4 de pueblos y nacionalidades.pdf procesado, longitud texto: 1749


In [ ]:
import os

print(os.listdir("/content/drive/MyDrive/rag_project/texts"))

['ejemplo 2 de pueblos y nacionalidades.txt', 'ejemplo pueblos y nacionalidades.txt', 'EJEMPLO 1 DE PUEBLOS Y NACIONALIDADES.txt', 'EJEMPLO 2 DE PUEBLOS Y NACIONALIDADES.txt', 'EJEMPLO 3 DE PUEBLOS Y NACIONALIDADES.txt', 'ejemplo 4 de pueblos y nacionalidades.txt']


In [ ]:
print("✅ PDFs convertidos a TXT correctamente.")

✅ PDFs convertidos a TXT correctamente.


## Cargar documentos

In [ ]:
import glob

input_folder = "/content/drive/MyDrive/rag_project/texts"

def get_txt_paths(folder_path):
    return glob.glob(f"{folder_path}/*.txt")

txt_files = get_txt_paths(input_folder)
print(txt_files)

['/content/drive/MyDrive/rag_project/texts/ejemplo 2 de pueblos y nacionalidades.txt', '/content/drive/MyDrive/rag_project/texts/ejemplo pueblos y nacionalidades.txt', '/content/drive/MyDrive/rag_project/texts/EJEMPLO 1 DE PUEBLOS Y NACIONALIDADES.txt', '/content/drive/MyDrive/rag_project/texts/EJEMPLO 2 DE PUEBLOS Y NACIONALIDADES.txt', '/content/drive/MyDrive/rag_project/texts/EJEMPLO 3 DE PUEBLOS Y NACIONALIDADES.txt', '/content/drive/MyDrive/rag_project/texts/ejemplo 4 de pueblos y nacionalidades.txt']


In [ ]:
documents = []

for i, file_path in enumerate(sorted(txt_files)):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    title = metadata.iloc[i]['titulo']
    lengua = metadata.iloc[i]['lengua']
    ubicacion = metadata.iloc[i]['ubicacion']
    meta = {
        'file_path': file_path.replace(input_folder, ''),
        'title': title,
        'lengua': lengua,
        'ubicacion': ubicacion
    }
    documents.append({
        'content': content,
        'meta': meta
    })

In [ ]:
print(f"✅ Documentos cargados: {len(documents)}")
print(documents[0])

✅ Documentos cargados: 6
{'content': 'NACIONALIDADES Y PUEBLOS ORIGINARIOS NACIONALIDAD SHUAR.\n\nCantón Macas Prodncia de Moron Samiago Ecuador\n\nCERTIFICADO DE PERTENENCIA\n\nLa suscrita señora, LUCÍA NANTIP JAMANOH, con códula de ciudadanía 1402987643, en\ncalidad de WINARU DE LAS NACIONALIDADES Y PUEBLOS ORIGINARIOS DE LA,\nNACIONALIDAD SHUAR. SHUARMAKA ICHIAMU NUWIAR, En uso de sus atribuciones\nestatutarias vigentes y en forma legal, concede el siguiente\n\nCERTIFICA\n\n¡Conocer al joven: KUNKU KUNTUI DAVID JAVIER, portador de La códula do identidad N*.\n140385021-7, de nacionalidad shuar, hablante detidioma Shuar Chicham, domiciliado\n¡na comunidad Taisha Yuwint, parroquía Talsha, cantón Taisha, provincia de Morona\nSantiago.\n\n¡Sus orígenos y ascendencia pertenecen a esta nacionalidad, en consecuencia, el\nmencionado joven forma parte legítima de las nacionalidades indigenas de la Amazonía\necuatoriana, por herencia ancestral y derecho propio,\n\n \n\nRazón pora cual CERTIFIC

## Crear nuevamente documentos de haystack con nuevos ids.

In [ ]:
import os

txt_folder = "/content/drive/MyDrive/rag_project/texts"
txt_files = sorted([f for f in os.listdir(txt_folder) if f.endswith('.txt')])

print(f"Archivos TXT encontrados: {len(txt_files)}")

for i, filename in enumerate(txt_files[:3]):
    path = os.path.join(txt_folder, filename)
    with open(path, 'r', encoding='utf-8') as f:
        content = f.read(500)  # primeros 500 caracteres
    print(f"Archivo {i}: {filename}\nContenido (primeros 500 chars):\n{content}\n---")

Archivos TXT encontrados: 6
Archivo 0: EJEMPLO 1 DE PUEBLOS Y NACIONALIDADES.txt
Contenido (primeros 500 chars):
NACIONALIDADES Y PUEBLOS ORIGINARIOS NACIONALIDAD SHUAR.

Cantón Macas Prodncia de Moron Samiago Ecuador

CERTIFICADO DE PERTENENCIA

La suscrita señora, LUCÍA NANTIP JAMANOH, con códula de ciudadanía 1402987643, en
calidad de WINARU DE LAS NACIONALIDADES Y PUEBLOS ORIGINARIOS DE LA,
NACIONALIDAD SHUAR. SHUARMAKA ICHIAMU NUWIAR, En uso de sus atribuciones
estatutarias vigentes y en forma legal, concede el siguiente

CERTIFICA

¡Conocer al joven: KUNKU KUNTUI DAVID JAVIER, portador de La códula d
---
Archivo 1: EJEMPLO 2 DE PUEBLOS Y NACIONALIDADES.txt
Contenido (primeros 500 chars):
NACIONALIDADES Y PUEBLOS ORIGINARIOS PUEBLO AFOECUATORIANO

CERTIFICADO DE PERTENENCIA

La suscrita señora, ANGÉLICA MOREIRA QUIÑÓNEZ, con cédula do ciudadanía
10804562391, en calidad de REPRESENTANTE LEGÍTIMA DEL PUEBLO AFROECUATORIANO
DE LA REGIÓN COSTA. EN USO DE SUS ATRIBUCIONES COMUNITARIAS 

In [ ]:
for i, doc in enumerate(documents[:3]):
    print(f"Doc {i} contenido (primeros 500 chars):\n{doc['content'][:500]}\n---")

Doc 0 contenido (primeros 500 chars):
NACIONALIDADES Y PUEBLOS ORIGINARIOS NACIONALIDAD SHUAR.

Cantón Macas Prodncia de Moron Samiago Ecuador

CERTIFICADO DE PERTENENCIA

La suscrita señora, LUCÍA NANTIP JAMANOH, con códula de ciudadanía 1402987643, en
calidad de WINARU DE LAS NACIONALIDADES Y PUEBLOS ORIGINARIOS DE LA,
NACIONALIDAD SHUAR. SHUARMAKA ICHIAMU NUWIAR, En uso de sus atribuciones
estatutarias vigentes y en forma legal, concede el siguiente

CERTIFICA

¡Conocer al joven: KUNKU KUNTUI DAVID JAVIER, portador de La códula d
---
Doc 1 contenido (primeros 500 chars):
NACIONALIDADES Y PUEBLOS ORIGINARIOS PUEBLO AFOECUATORIANO

CERTIFICADO DE PERTENENCIA

La suscrita señora, ANGÉLICA MOREIRA QUIÑÓNEZ, con cédula do ciudadanía
10804562391, en calidad de REPRESENTANTE LEGÍTIMA DEL PUEBLO AFROECUATORIANO
DE LA REGIÓN COSTA. EN USO DE SUS ATRIBUCIONES COMUNITARIAS y conforme ala
normativa igento, concede el siguiente:

 

CERTIFICA

¡Conocer ala señorita: PALACIOS MONTENEGRO ESTEFANÍA

# Vectorizar y buscar similitud

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

vectorizer = TfidfVectorizer()
doc_texts = [doc['content'] for doc in documents]
doc_vectors = vectorizer.fit_transform(doc_texts)
print(f"Vectorización completada para {len(documents)} documentos.")

Vectorización completada para 6 documentos.


In [ ]:
def buscar_similitud(query, top_k=5):
    query_vec = vectorizer.transform([query])
    similitudes = cosine_similarity(query_vec, doc_vectors).flatten()
    idx_top = np.argsort(similitudes)[::-1][:top_k]
    resultados = []
    for idx in idx_top:
        resultados.append({
            'content': documents[idx]['content'],
            'meta': documents[idx]['meta'],
            'score': similitudes[idx]
        })
    return resultados

query = "lenguas indígenas amazónicas"
resultados = buscar_similitud(query)
for r in resultados:
    print(f"Score: {r['score']:.4f}")
    print(f"Título: {r['meta']['title']}")
    print(f"Contenido: {r['content'][:200]}...\n")

Score: 0.0523
Título: NACIONALIDAD CHACHI
Contenido: NACIONALIDADES Y PUEBLOS ORIGINARIOS

CAMPESINOS AMAZÓNICOS DE NAPO “NAPOCA”
Resolución N' SGDPM-ORCPN-2022-0067.R. 28 de octubre de 2022
Cantón Tena - Provincia de Napo — Ecuador

CERTIFICADO DE PERT...

Score: 0.0000
Título: NACIONALIDAD A´I KOFÁN
Contenido: CHINCHAYSUYU JATUN AYLLU LLAKTAKUNAPAK TANTARI

FEDERACIÓN DE LOS PUEBLOS KICHWA DE LA SIERRA NORTE DEL ECUADOR
FILIAL ECUARUNARI - CONAIE
Acuerdo Miniteral N 011 CODENPE - 12 de Noviembre el 2003
La ...

Score: 0.0000
Título: NACIONALIDAD AWÁ
Contenido: CHINCHAYSUYU JATUN AYLLU LLAKTAKUNAPAK TANTARI
La Federación de los Pueblos Kichwa dela Sierra Norte del Ecuador, y en su nombre,
Dolores Pichamba Yumbo, en cidad de Presidenta y Representante Legal d...

Score: 0.0000
Título: NACIONALIDAD EPERARA
Contenido: CHINCHAYSUYU JATUN AYLLU LLAKTAKUNAPAK TANTARI
La Federación delos Pueblos Kichwa dela Siera Centro del Ecuador, y en su nombre,
Patricio Quishpe Tanday, en calidad de Pres

# Guardar documentos

In [ ]:
import pickle

# Guardar la lista documents en un archivo pickle
output_path = "/content/drive/MyDrive/rag_project/documents.pkl"

with open(output_path, "wb") as f:
    pickle.dump(documents, f)

print(f"📦 Documents guardados en {output_path}")

📦 Documents guardados en /content/drive/MyDrive/rag_project/documents.pkl


In [ ]:
import pickle

input_path = "/content/drive/MyDrive/rag_project/documents.pkl"

with open(input_path, "rb") as f:
    documents = pickle.load(f)

print(f"📂 Documents cargados, total: {len(documents)}")
print(documents[0])  # Para verificar el contenido

📂 Documents cargados, total: 6
{'content': 'NACIONALIDADES Y PUEBLOS ORIGINARIOS NACIONALIDAD SHUAR.\n\nCantón Macas Prodncia de Moron Samiago Ecuador\n\nCERTIFICADO DE PERTENENCIA\n\nLa suscrita señora, LUCÍA NANTIP JAMANOH, con códula de ciudadanía 1402987643, en\ncalidad de WINARU DE LAS NACIONALIDADES Y PUEBLOS ORIGINARIOS DE LA,\nNACIONALIDAD SHUAR. SHUARMAKA ICHIAMU NUWIAR, En uso de sus atribuciones\nestatutarias vigentes y en forma legal, concede el siguiente\n\nCERTIFICA\n\n¡Conocer al joven: KUNKU KUNTUI DAVID JAVIER, portador de La códula do identidad N*.\n140385021-7, de nacionalidad shuar, hablante detidioma Shuar Chicham, domiciliado\n¡na comunidad Taisha Yuwint, parroquía Talsha, cantón Taisha, provincia de Morona\nSantiago.\n\n¡Sus orígenos y ascendencia pertenecen a esta nacionalidad, en consecuencia, el\nmencionado joven forma parte legítima de las nacionalidades indigenas de la Amazonía\necuatoriana, por herencia ancestral y derecho propio,\n\n \n\nRazón pora cual CE